In [ ]:
!pip install qiskit==1.4.2
!pip install qiskit_machine_learning==0.8.2
!pip install qiskit_algorithms==0.3.0
!pip install openpyxl
!pip install XlsxWriter
!pip install pylatexenc


In [ ]:
import warnings, time
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityStatevectorKernel

warnings.filterwarnings("ignore")
SEED = 12345
np.random.seed(SEED)

# ====== CONFIG ======
NUM_REPS       = 30
TRAIN_SIZE     = 500
TEST_SIZE      = 200
FEATURE_DIM    = 2
C_LIST         = [0.1, 1, 10]
SCALES         = np.logspace(-3, 0.3, 8)
CV_CLASSIC_K   = 3
CV_QUANT_K     = 3


def load_dataset():
    Xtr, ytr, Xts, yts = ad_hoc_data(
        training_size=TRAIN_SIZE,
        test_size=TEST_SIZE,
        n=FEATURE_DIM,
        gap=0.3,
        one_hot=False,
        include_sample_total=False
    )
    # recode labels {0→-1,1→+1}
    ytr = np.where(ytr == 0, -1, 1)
    yts = np.where(yts == 0, -1, 1)
    return Xtr, ytr, Xts, yts


def get_classical_models():
    gammas = np.logspace(-3, 1, 5)
    return {
        "LogReg":   ( LogisticRegression(max_iter=1000), {"C": C_LIST} ),
        "SVM-lin":  ( SVC(kernel="linear"),              {"C": C_LIST} ),
        "SVM-poly": ( SVC(kernel="poly"),   {"degree":[3,4,5],"C":C_LIST,"coef0":[0,1]} ),
        "SVM-rbf":  ( SVC(kernel="rbf"),    {"C":C_LIST,"gamma":gammas} ),
    }


def build_feature_maps():
    maps = []
    # ZFeatureMap reps 1,2
    for r in [1,2]:
        maps.append((f"Z-reps{r}", ZFeatureMap(FEATURE_DIM, reps=r)))
    # ZZFeatureMap reps 1,2 con entanglement linear y full
    for ent in ["linear", "full"]:
        for r in [1,2]:
            maps.append((f"ZZ-{ent}-r{r}", ZZFeatureMap(FEATURE_DIM, reps=r, entanglement=ent)))
    return maps


if __name__ == "__main__":
    classical_records = []
    quantum_records   = []

    for rep in range(NUM_REPS):
        print(f"--- Repetition {rep+1}/{NUM_REPS} ---")
        Xtr, ytr, Xts, yts = load_dataset()


        for name, (clf, grid) in get_classical_models().items():
            t0 = time.time()
            gs = GridSearchCV(clf, grid, cv=CV_CLASSIC_K, scoring="accuracy", n_jobs=-1)
            gs.fit(Xtr, ytr)
            best = gs.best_estimator_
            ypred = best.predict(Xts)
            t_eval = time.time() - t0

            classical_records.append({
                "model":    name,
                "accuracy": accuracy_score(yts, ypred),
                "f1":       f1_score(yts, ypred),
                "auc":      roc_auc_score(yts, ypred),
                "time_s":   round(t_eval,2)
            })

        # ----- Kernels cuánticos -----
        # pre-escalado base [0, π]
        scaler = MinMaxScaler((0, np.pi))
        Xtr0, Xts0 = scaler.fit_transform(Xtr), scaler.transform(Xts)

        for fmap_name, fmap in build_feature_maps():
            for idx, scale in enumerate(SCALES, start=1):
                t0 = time.time()

                # aplicar escala (numérica)
                Xtr_s = Xtr0 * scale
                Xts_s = Xts0 * scale

                # kernel exacto statevector
                qk = FidelityStatevectorKernel(feature_map=fmap)
                # parche interno para evitar el setter de num_qubits
                qk._validate_input = lambda x, y=None: (x, y)

                Ktr = qk.evaluate(Xtr_s)
                Kts = qk.evaluate(Xts_s, Xtr_s)

                # GridSearchCV para C
                svc = GridSearchCV(
                    SVC(kernel="precomputed"),
                    {"C": C_LIST},
                    cv=CV_QUANT_K,
                    scoring="accuracy"
                )
                svc.fit(Ktr, ytr)
                ypred_q = svc.predict(Kts)
                t_eval = time.time() - t0

                quantum_records.append({
                    # nombre con ordinal de escala, NO el valor real
                    "feature_map": f"{fmap_name}-scale{idx}",
                    # guardamos la escala real en columna aparte
                    "scale":       scale,
                    "accuracy":    accuracy_score(yts, ypred_q),
                    "f1":          f1_score(yts, ypred_q),
                    "auc":         roc_auc_score(yts, ypred_q),
                    "time_s":      round(t_eval,2)
                })

    # ----- Agregación -----
    df_cl = pd.DataFrame(classical_records)
    df_q  = pd.DataFrame(quantum_records)

    cl_agg = df_cl.groupby("model").agg(
        acc_mean = ("accuracy", "mean"),
        acc_std  = ("accuracy", "std"),
        f1_mean  = ("f1",       "mean"),
        f1_std   = ("f1",       "std"),
        time_s = ("time_s", "mean")
    ).reset_index().sort_values("acc_mean", ascending=False)

    q_agg = df_q.groupby("feature_map").agg(
        acc_mean = ("accuracy", "mean"),
        acc_std  = ("accuracy", "std"),
        f1_mean  = ("f1",       "mean"),
        f1_std   = ("f1",       "std"),
        scale = ("scale", "mean"),
        time_s = ("time_s", "mean")

    ).reset_index().sort_values("acc_mean", ascending=False)

    print("\n Clásicos (mean ± std):")
    print(cl_agg.to_string(index=False))

    print("\n Cuánticos (mean ± std):")
    print(q_agg.to_string(index=False))

    # Guardar en Excel
    with pd.ExcelWriter("Resultados_ADHOC_Aggregate.xlsx") as writer:
        cl_agg.to_excel(writer, sheet_name="Clasicos", index=False)
        q_agg.to_excel(writer, sheet_name="Cuanticos", index=False)

    print("\n Resultados guardados en Resultados_ADHOC_Aggregate.xlsxv")
